In [1]:
import category_encoders as ce
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

In [2]:
data = fetch_california_housing(as_frame=True)
df = data.frame

In [3]:
df['region'] = pd.cut(df['Longitude'], bins=3, labels=['West', 'Central', 'East'])
df['density'] = pd.cut(df['AveOccup'], bins=3, labels=['Low', 'Medium', 'High'])

In [4]:
# Відбираємо тільки категоріальні фічі + цільову змінну
cat_columns = ['region', 'density']
df = df[cat_columns + ['MedHouseVal']].dropna()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['MedHouseVal']), 
                                                    df['MedHouseVal'], test_size=0.2, random_state=42)

### 1️⃣ One-Hot Encoding (OHE) ###

- OHE створює окрему колонку для кожного унікального значення категоріальної змінної (dummy-кодування).
- Підходить, коли кількість категорій невелика, інакше матриця стає занадто великою.

In [6]:
ohe_encoder = ce.OneHotEncoder(cols=cat_columns, use_cat_names=True)
X_train_ohe = ohe_encoder.fit_transform(X_train)
X_test_ohe = ohe_encoder.transform(X_test)
X_train_ohe.head()

,region_West,region_Central,region_East,density_Low,density_Medium,density_High
14196,0,0,1,1,0,0
8267,0,1,0,1,0,0
17445,0,1,0,1,0,0
14265,0,0,1,1,0,0
2271,0,1,0,1,0,0


### 2️⃣ Target Encoding ###

- Замінює категорію середнім значенням цільової змінної для цієї категорії.
- Використовується для моделей, чутливих до порядку чисел (наприклад, лінійна регресія).
- Може викликати виток даних (data leakage), тому важливо правильно крос-валідувати.

In [7]:
target_encoder = ce.TargetEncoder(cols=cat_columns)
X_train_te = target_encoder.fit_transform(X_train, y_train)
X_test_te = target_encoder.transform(X_test)
X_train_te.head()

,region,density
14196,1.647410,2.071987
8267,2.137346,2.071987
17445,2.137346,2.071987
14265,1.647410,2.071987
2271,2.137346,2.071987


### 3️⃣ Ordinal Encoding ###

- Замінює кожну категорію унікальним числом (1, 2, 3...).
- Добре працює для моделей, які можуть сприймати порядок (наприклад, дерева рішень).
- Якщо порядок категорій немає сенсу, модель може неправильно інтерпретувати відстані між значеннями.

In [8]:
ordinal_encoder = ce.OrdinalEncoder(cols=cat_columns)
X_train_oe = ordinal_encoder.fit_transform(X_train)
X_test_oe = ordinal_encoder.transform(X_test)
X_train_oe.head()

,region,density
14196,3,1
8267,2,1
17445,2,1
14265,3,1
2271,2,1


### 4️⃣ Binary Encoding ###

- Кодує кожну категорію у двійковий код, кожен біт розміщується у окремій колонці.
- Зменшує кількість колонок у порівнянні з One-Hot Encoding.
- Добре працює при великій кількості унікальних категорій.

In [9]:
binary_encoder = ce.BinaryEncoder(cols=cat_columns)
X_train_be = binary_encoder.fit_transform(X_train)
X_test_be = binary_encoder.transform(X_test)
X_train_be.head()

,region_0,region_1,density_0,density_1
14196,1,1,0,1
8267,1,0,0,1
17445,1,0,0,1
14265,1,1,0,1
2271,1,0,0,1


### 5️⃣ Hashing Encoder ###

- ✅ Коли у датасеті багато унікальних категорій (наприклад, ID користувачів, домени сайтів).
- ✅ Коли немає потреби у зворотному декодуванні (бо хеш-функція необоротна).
- ✅ Коли важливий розмір даних (це компактніше за One-Hot Encoding).
- ✅ Використовується у градієнтному бустингу (XGBoost, CatBoost) та нейромережах.

- 🚫 Коли НЕ варто використовувати:
- ❌ Якщо потрібно інтерпретувати значення (бо неможливо відновити оригінальну категорію).
- ❌ Якщо даних мало, а колізії можуть суттєво спотворити інформацію.

In [12]:
hashing_enc = ce.HashingEncoder(cols=['region', 'density'], n_components=8)
X_train_hsh = hashing_enc.fit_transform(X_train)
X_test_hsh = hashing_enc.transform(X_test)
X_train_hsh.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
14196,0,0,0,0,0,0,1,1
8267,0,0,1,0,0,0,0,1
17445,0,0,1,0,0,0,0,1
14265,0,0,0,0,0,0,1,1
2271,0,0,1,0,0,0,0,1
